In [ ]:
from cvzone.FaceDetectionModule import FaceDetector
from time import time
import cvzone
import cv2

########### PARAMETERS ###########
classID = 1 # 0 is fake and 1 is real
outputFolderPath = 'Dataset/DataCollect'
confidence = 0.8
blurThreshold = 100 # large is more focus
save = False

debug = True
offsetPercentageW = 10
offsetPercentageH = 20
camWidth, camHeight = 640, 480
floatingPoint = 6
######################
#cap = cv2.VideoCapture("http://192.168.43.195:8080/video")
cap = cv2.VideoCapture(2)
cap.set(3,camWidth)
cap.set(4,camHeight)
#cap.set(10,150)

detector = FaceDetector()

while True:
    success,img = cap.read()
    #img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    #img = cv2.resize(img, (480, 640))
    imgOut = img.copy()
    img,bboxs = detector.findFaces(img, draw = False)
    
    listBlur = []  # True, False values indicating if the faces are blur or not
    listInfo = []  # The normalized values and the class name for the label text file
    if bboxs:
        #Bounding box info: "id","bbox","score","center"
        for bbox in bboxs:
            x, y, w, h = bbox["bbox"]
            score = bbox["score"][0]
            #print(x,y,w,h)
            
            #........ Check the score that it is human face ......#
            if score > confidence:
                #........ Adding offset to face detected ......#
                offsetW = (offsetPercentageW / 100) * w
                x = int(x - offsetW)
                w = int(w + offsetW * 2)
                offsetH = (offsetPercentageH / 100) * h
                y = int(y - offsetH * 3)
                h = int(h + offsetH * 3.5)

                #........ To avoid values below zero ......#
                if x < 0: x = 0
                if y < 0: y = 0
                if w < 0: w = 0
                if h < 0: h = 0
                #........ Finding blurriness in the face detected ......#
                imgFace = img[y:y+h,x:x+w]
                cv2.imshow("Face",imgFace)
                blurValue = int(cv2.Laplacian(imgFace,cv2.CV_64F).var())
                if blurValue > blurThreshold:
                    listBlur.append(True)
                else:
                    listBlur.append(False)
                
                #........ Normalization of values ......#
                ih, iw, _ = img.shape
                xc, yc = x + w/2, y + h/2
                xcn, ycn = round(xc/iw,floatingPoint),round(yc/ih,floatingPoint)
                wn, hn = round(w/iw,floatingPoint),round(h/ih,floatingPoint)
                #print(xcn,ycn,wn,hn)
                
                #........ To avoid values above 1 ......#
                if xcn > 1: xcn = 1
                if ycn > 1: ycn = 1
                if wn > 1: wn = 1
                if hn > 1: hn = 1
                    
                listInfo.append(f"{classID} {xcn} {ycn} {wn} {hn}\n")

                #........ Drawing ......#
                cv2.rectangle(imgOut,(x,y,w,h),(255,0,0),3)
                cvzone.putTextRect(imgOut,f'Score:{int(score*100)}% Blur: {blurValue}',(x, y-20),
                                  scale=2,thickness=3)
                if debug:
                    cv2.rectangle(img,(x,y,w,h),(255,0,0),3)
                    cvzone.putTextRect(img,f'Score:{int(score*100)}% Blur: {blurValue}',(x, y-20),
                                  scale=2,thickness=3)
                
        #........ To save ......#
        if save:
            #print(listBlur,all(listBlur))
            if all(listBlur) and listBlur != []:
                
                #........ To save image ......#
                timeNow = time()
                timeNow = str(timeNow).split('.')
                timeNow = timeNow[0] + timeNow[1]
                print(timeNow)
                cv2.imwrite(f"{outputFolderPath}/{timeNow}.jpg",img)
                
                #........ To save image ......#
                for info in listInfo:
                    f = open(f"{outputFolderPath}/{timeNow}.txt",'a')
                    f.write(info)
                    f.close()
                
         
    cv2.imshow("Image",imgOut)
    cv2.waitKey(1)

2024-04-18 12:13:41.367613: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-18 12:13:41.407561: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-18 12:13:41.407601: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-18 12:13:41.408688: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-18 12:13:41.414876: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-18 12:13:41.415587: I tensorflow/core/platform/cpu_feature_guard.cc:1